# EK80 Example

### Setting up project root for module imports in Jupyter Notebook

In [1]:
import os

notebook_path = os.path.abspath("analysis_ek80_dataset_T133707.ipynb")
notebooks_folder_path = os.path.abspath(os.path.join(notebook_path, os.pardir))

project_root = os.path.abspath(os.path.join(notebook_path, os.pardir, os.pardir))

import sys

sys.path.append(project_root)

**Specifies the path to the test data utilized in this notebook:**

In [2]:
oceanstream_ek80_path =  os.path.join(notebooks_folder_path, "test_data", "oceanstream_ek80_T133707")

### Imports

In [3]:
import gc
import os
import urllib.request

import echopype.visualize as epviz
import fsspec
import matplotlib.pyplot as plt
import numpy as np

from oceanstream.L0_unprocessed_data import raw_handler, ensure_time_continuity
from oceanstream.L2_calibrated_data import (
    sv_computation,
)

In [4]:
# Load the memory_profiler extension to enable memory usage profiling in the notebook
%load_ext memory_profiler

### Downloads the test files

In [5]:
# Initialize the S3 filesystem using anonymous access
fs = fsspec.filesystem('s3', anon=True)

bucket = "noaa-wcsd-pds"
rawdirpath = "data/raw/Pisces/PC1905/EK80"

# List files in the S3 directory
s3_files = fs.glob(f"{bucket}/{rawdirpath}/*")

# Filter the files based on your criteria (e.g., containing a specific string)
target_files = [s3path for s3path in s3_files if any([f"D2019{dtstr}" in s3path for dtstr in ['0925-T133707']])]

print(f"There are {len(target_files)} target files available")

There are 1 target files available


In [6]:
local_folder = oceanstream_ek80_path

# Create the local folder if it doesn't exist
os.makedirs(local_folder, exist_ok=True)

# Download the filtered files to the local folder
for s3path in target_files:
    file_name = os.path.basename(s3path)
    local_file_path = os.path.join(local_folder, file_name)
    with fs.open(s3path, 'rb') as s3_file, open(local_file_path, 'wb') as local_file:
        local_file.write(s3_file.read())

print(f"Downloaded {len(target_files)} target raw files to {local_folder}")

Downloaded 1 target raw files to /Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80_T133707


### Files manipulation

**File finder**

In [7]:
%%time

# Displays all `.raw` files from the "reduced_T210646" directory
%memit raw_files_path = raw_handler.file_finder(oceanstream_ek80_path, file_type='raw')
display(raw_files_path)

peak memory: 265.28 MiB, increment: 0.03 MiB


['/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80_T133707/D20190925-T133707.raw']

CPU times: user 101 ms, sys: 14.5 ms, total: 115 ms
Wall time: 662 ms


**File integrity checker**

In [8]:
%%time

# Verifies if the provided echo sounder files are
# readable by echopype and extracts
# essential metadata such as the campaign ID, date of measurement,
# and sonar model.
raw_files_ck = []
for file_path in raw_files_path:
    %memit raw_files_ck.append(raw_handler.file_integrity_checking(file_path))
    
raw_files_ck

peak memory: 1390.33 MiB, increment: 1125.04 MiB
CPU times: user 10.7 s, sys: 859 ms, total: 11.6 s
Wall time: 12.1 s


[{'file_path': '/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80_T133707/D20190925-T133707.raw',
  'campaign_id': 'D20190925',
  'date': datetime.datetime(2019, 9, 25, 13, 37, 7),
  'sonar_model': 'EK80',
  'file_integrity': True,
  'use_swap': False}]

**Checked raw files reader**

In [9]:
%%time

# Reads multiple raw echo sounder files and returns a list of Datasets.
%memit sept2019_ds = raw_handler.read_raw_files(raw_files_ck)
sept2019_ds[0]

peak memory: 1386.39 MiB, increment: 931.71 MiB
CPU times: user 4.73 s, sys: 697 ms, total: 5.42 s
Wall time: 5.99 s


<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

### NetCDF files obtained from raw files

**Raw file converter to 'nc' or 'zarr'**

In [10]:
%%time

%memit raw_handler.convert_raw_files(raw_files_ck, oceanstream_ek80_path, save_file_type='nc')

peak memory: 1708.45 MiB, increment: 1172.88 MiB
CPU times: user 8.33 s, sys: 841 ms, total: 9.17 s
Wall time: 9.85 s


In [11]:
%%time

%memit raw_handler.convert_raw_files(raw_files_ck, oceanstream_ek80_path, save_file_type='zarr')

peak memory: 1752.27 MiB, increment: 972.46 MiB
CPU times: user 6.94 s, sys: 960 ms, total: 7.9 s
Wall time: 6.84 s


**File finder**

In [12]:
%%time

%memit nc_files_path = raw_handler.file_finder(oceanstream_ek80_path, file_type='nc')
nc_files_path

peak memory: 808.59 MiB, increment: 0.00 MiB
CPU times: user 107 ms, sys: 40.6 ms, total: 148 ms
Wall time: 692 ms


['/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80_T133707/D20190925-T133707.nc']

**File integrity checker**

In [13]:
%%time

nc_files_ck = []
for file_path in nc_files_path:
    %memit nc_files_ck.append(raw_handler.file_integrity_checking(file_path))
    
nc_files_ck

peak memory: 822.05 MiB, increment: 13.46 MiB
CPU times: user 331 ms, sys: 61 ms, total: 392 ms
Wall time: 923 ms


[{'file_path': '/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80_T133707/D20190925-T133707.nc',
  'campaign_id': 'D20190925',
  'date': datetime.datetime(2019, 9, 25, 13, 37, 7),
  'sonar_model': 'EK80',
  'file_integrity': True}]

**Processed files reader**

In [14]:
%%time

%memit ds = raw_handler.read_processed_files(nc_files_path)

peak memory: 645.80 MiB, increment: 11.27 MiB
CPU times: user 376 ms, sys: 74.1 ms, total: 450 ms
Wall time: 982 ms


In [15]:
ds = ds[0]

### Sv (Volume backscattering strength) computation

In [16]:
%%time

%memit sv = sv_computation.compute_sv(ds, waveform_mode="CW", encode_mode="power")
sv

peak memory: 1491.18 MiB, increment: 845.39 MiB
CPU times: user 1.24 s, sys: 441 ms, total: 1.68 s
Wall time: 2.23 s


<xarray.Dataset>
Dimensions:                        (channel: 4, ping_time: 4234,
                                    range_sample: 910, filenames: 1)
Coordinates:
  * channel                        (channel) <U27 'GPT 0090720346b4-1 ES38B' ...
  * ping_time                      (ping_time) datetime64[ns] 2019-09-25T13:3...
  * range_sample                   (range_sample) int64 0 1 2 3 ... 907 908 909
  * filenames                      (filenames) int64 0
Data variables: (12/24)
    Sv                             (channel, ping_time, range_sample) float64 ...
    echo_range                     (channel, ping_time, range_sample) float64 ...
    frequency_nominal              (channel) float64 ...
    sound_speed                    float64 1.51e+03
    sound_absorption               (channel) float64 0.009878 ... 0.05316
    temperature                    float64 10.0
    ...                             ...
    beamwidth_athwartship          (channel) float64 ...
    impedance_transducer           (channel) int64 75 75 75 75
    impedance_transceiver          (channel) int64 ...
    receiver_sampling_frequency    (channel) float64 ...
    source_filenames               (filenames) <U119 '/Users/simedroniraluca/...
    water_level                    float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.1.dev2337+g364c06a
    processing_time:              2023-11-21T15:58:42Z
    processing_function:          calibrate.compute_Sv

In [17]:
np.nanmax(sv.echo_range.values)

21.9643488

**'GPT 0090720346b4-1 ES38B' Channel**

In [18]:
selected_channel_38 = 'GPT 0090720346b4-1 ES38B'

In [19]:
selected_channel_data_38 = sv.sel(channel=selected_channel_38)
selected_channel_data_38

<xarray.Dataset>
Dimensions:                        (ping_time: 4234, range_sample: 910,
                                    filenames: 1)
Coordinates:
    channel                        <U27 'GPT 0090720346b4-1 ES38B'
  * ping_time                      (ping_time) datetime64[ns] 2019-09-25T13:3...
  * range_sample                   (range_sample) int64 0 1 2 3 ... 907 908 909
  * filenames                      (filenames) int64 0
Data variables: (12/24)
    Sv                             (ping_time, range_sample) float64 nan ... nan
    echo_range                     (ping_time, range_sample) float64 0.0 ... nan
    frequency_nominal              float64 ...
    sound_speed                    float64 1.51e+03
    sound_absorption               float64 0.009878
    temperature                    float64 10.0
    ...                             ...
    beamwidth_athwartship          float64 ...
    impedance_transducer           int64 75
    impedance_transceiver          int64 ...
    receiver_sampling_frequency    float64 ...
    source_filenames               (filenames) <U119 '/Users/simedroniraluca/...
    water_level                    float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.1.dev2337+g364c06a
    processing_time:              2023-11-21T15:58:42Z
    processing_function:          calibrate.compute_Sv

In [20]:
selected_channel_data_38.echo_range.max()

<xarray.DataArray 'echo_range' ()>
array(21.9401856)
Coordinates:
    channel  <U27 'GPT 0090720346b4-1 ES38B'

In [21]:
selected_channel_data_38.ping_time

<xarray.DataArray 'ping_time' (ping_time: 4234)>
array(['2019-09-25T13:37:07.911682048', '2019-09-25T13:37:08.131695104',
       '2019-09-25T13:37:08.348708864', ..., '2019-09-25T13:52:20.565571072',
       '2019-09-25T13:52:20.782582784', '2019-09-25T13:52:20.998596096'],
      dtype='datetime64[ns]')
Coordinates:
    channel    <U27 'GPT 0090720346b4-1 ES38B'
  * ping_time  (ping_time) datetime64[ns] 2019-09-25T13:37:07.911682048 ... 2...

In [22]:
selected_channel_data_38.ping_time[0]

<xarray.DataArray 'ping_time' ()>
array('2019-09-25T13:37:07.911682048', dtype='datetime64[ns]')
Coordinates:
    channel    <U27 'GPT 0090720346b4-1 ES38B'
    ping_time  datetime64[ns] 2019-09-25T13:37:07.911682048

In [23]:
selected_channel_data_38.sel(ping_time='2019-09-25T13:37:07.911682048').echo_range.values

array([ 0.       ,  0.0966528,  0.1933056,  0.2899584,  0.3866112,
        0.483264 ,  0.5799168,  0.6765696,  0.7732224,  0.8698752,
        0.966528 ,  1.0631808,  1.1598336,  1.2564864,  1.3531392,
        1.449792 ,  1.5464448,  1.6430976,  1.7397504,  1.8364032,
        1.933056 ,  2.0297088,  2.1263616,  2.2230144,  2.3196672,
        2.41632  ,  2.5129728,  2.6096256,  2.7062784,  2.8029312,
        2.899584 ,  2.9962368,  3.0928896,  3.1895424,  3.2861952,
        3.382848 ,  3.4795008,  3.5761536,  3.6728064,  3.7694592,
        3.866112 ,  3.9627648,  4.0594176,  4.1560704,  4.2527232,
        4.349376 ,  4.4460288,  4.5426816,  4.6393344,  4.7359872,
        4.83264  ,  4.9292928,  5.0259456,  5.1225984,  5.2192512,
        5.315904 ,  5.4125568,  5.5092096,  5.6058624,  5.7025152,
        5.799168 ,  5.8958208,  5.9924736,  6.0891264,  6.1857792,
        6.282432 ,  6.3790848,  6.4757376,  6.5723904,  6.6690432,
        6.765696 ,  6.8623488,  6.9590016,  7.0556544,  7.1523

In [24]:
selected_channel_data_38.sel(ping_time='2019-09-25T13:37:07.911682048').Sv.values

array([          nan,           nan,           nan,           nan,
       -176.57134078,  -43.2773642 ,  -44.09858195,  -45.86638936,
        -39.55579534,  -32.73570743,  -35.58331093,  -46.48356789,
        -42.73484438,  -46.36643964,  -57.67872377,  -55.74361773,
        -68.17951767,  -59.50977468,  -80.63005212,  -62.34336183,
        -67.23704702,  -66.08533642,  -63.17636415,  -57.09579345,
        -58.78849281,  -64.41867504,  -71.31502787,  -88.85946463,
        -75.12967935,  -73.83247493,  -68.62168121,  -67.49234946,
        -66.30439168,  -66.32693703,  -63.07905485,  -58.64141955,
        -57.4818169 ,  -58.84712679,  -62.04305584,  -60.64874645,
        -58.02683505,  -60.15049702,  -79.51917635,  -63.94617243,
        -59.43232494,  -57.25218542,  -59.26341753,  -63.99589886,
        -65.92267717,  -67.43062619,  -72.12954925,  -71.55279299,
        -70.18035571,  -71.81023196,  -75.68969825,  -82.45358289,
        -78.97873455,  -70.46249761,  -66.4765598 ,  -66.51517

In [25]:
selected_channel_data_38.isel(ping_time=900).echo_range.values

array([ 0.       ,  0.0966528,  0.1933056,  0.2899584,  0.3866112,
        0.483264 ,  0.5799168,  0.6765696,  0.7732224,  0.8698752,
        0.966528 ,  1.0631808,  1.1598336,  1.2564864,  1.3531392,
        1.449792 ,  1.5464448,  1.6430976,  1.7397504,  1.8364032,
        1.933056 ,  2.0297088,  2.1263616,  2.2230144,  2.3196672,
        2.41632  ,  2.5129728,  2.6096256,  2.7062784,  2.8029312,
        2.899584 ,  2.9962368,  3.0928896,  3.1895424,  3.2861952,
        3.382848 ,  3.4795008,  3.5761536,  3.6728064,  3.7694592,
        3.866112 ,  3.9627648,  4.0594176,  4.1560704,  4.2527232,
        4.349376 ,  4.4460288,  4.5426816,  4.6393344,  4.7359872,
        4.83264  ,  4.9292928,  5.0259456,  5.1225984,  5.2192512,
        5.315904 ,  5.4125568,  5.5092096,  5.6058624,  5.7025152,
        5.799168 ,  5.8958208,  5.9924736,  6.0891264,  6.1857792,
        6.282432 ,  6.3790848,  6.4757376,  6.5723904,  6.6690432,
        6.765696 ,  6.8623488,  6.9590016,  7.0556544,  7.1523

**'GPT 009072058cb4-1 ES120-7C' Channel**

In [26]:
selected_channel_120 = 'GPT 009072058cb4-1 ES120-7C'

In [27]:
selected_channel_data_120 = sv.sel(channel=selected_channel_120)
selected_channel_data_120

<xarray.Dataset>
Dimensions:                        (ping_time: 4234, range_sample: 910,
                                    filenames: 1)
Coordinates:
    channel                        <U27 'GPT 009072058cb4-1 ES120-7C'
  * ping_time                      (ping_time) datetime64[ns] 2019-09-25T13:3...
  * range_sample                   (range_sample) int64 0 1 2 3 ... 907 908 909
  * filenames                      (filenames) int64 0
Data variables: (12/24)
    Sv                             (ping_time, range_sample) float64 nan ... ...
    echo_range                     (ping_time, range_sample) float64 0.0 ... ...
    frequency_nominal              float64 ...
    sound_speed                    float64 1.51e+03
    sound_absorption               float64 0.03779
    temperature                    float64 10.0
    ...                             ...
    beamwidth_athwartship          float64 ...
    impedance_transducer           int64 75
    impedance_transceiver          int64 ...
    receiver_sampling_frequency    float64 ...
    source_filenames               (filenames) <U119 '/Users/simedroniraluca/...
    water_level                    float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.1.dev2337+g364c06a
    processing_time:              2023-11-21T15:58:42Z
    processing_function:          calibrate.compute_Sv

In [28]:
selected_channel_data_120.echo_range.max()

<xarray.DataArray 'echo_range' ()>
array(21.9643488)
Coordinates:
    channel  <U27 'GPT 009072058cb4-1 ES120-7C'

In [29]:
selected_channel_data_120.isel(ping_time=0).echo_range.values

array([ 0.       ,  0.0241632,  0.0483264,  0.0724896,  0.0966528,
        0.120816 ,  0.1449792,  0.1691424,  0.1933056,  0.2174688,
        0.241632 ,  0.2657952,  0.2899584,  0.3141216,  0.3382848,
        0.362448 ,  0.3866112,  0.4107744,  0.4349376,  0.4591008,
        0.483264 ,  0.5074272,  0.5315904,  0.5557536,  0.5799168,
        0.60408  ,  0.6282432,  0.6524064,  0.6765696,  0.7007328,
        0.724896 ,  0.7490592,  0.7732224,  0.7973856,  0.8215488,
        0.845712 ,  0.8698752,  0.8940384,  0.9182016,  0.9423648,
        0.966528 ,  0.9906912,  1.0148544,  1.0390176,  1.0631808,
        1.087344 ,  1.1115072,  1.1356704,  1.1598336,  1.1839968,
        1.20816  ,  1.2323232,  1.2564864,  1.2806496,  1.3048128,
        1.328976 ,  1.3531392,  1.3773024,  1.4014656,  1.4256288,
        1.449792 ,  1.4739552,  1.4981184,  1.5222816,  1.5464448,
        1.570608 ,  1.5947712,  1.6189344,  1.6430976,  1.6672608,
        1.691424 ,  1.7155872,  1.7397504,  1.7639136,  1.7880

In [30]:
selected_channel_data_120.isel(ping_time=900).echo_range.values

array([ 0.       ,  0.0241632,  0.0483264,  0.0724896,  0.0966528,
        0.120816 ,  0.1449792,  0.1691424,  0.1933056,  0.2174688,
        0.241632 ,  0.2657952,  0.2899584,  0.3141216,  0.3382848,
        0.362448 ,  0.3866112,  0.4107744,  0.4349376,  0.4591008,
        0.483264 ,  0.5074272,  0.5315904,  0.5557536,  0.5799168,
        0.60408  ,  0.6282432,  0.6524064,  0.6765696,  0.7007328,
        0.724896 ,  0.7490592,  0.7732224,  0.7973856,  0.8215488,
        0.845712 ,  0.8698752,  0.8940384,  0.9182016,  0.9423648,
        0.966528 ,  0.9906912,  1.0148544,  1.0390176,  1.0631808,
        1.087344 ,  1.1115072,  1.1356704,  1.1598336,  1.1839968,
        1.20816  ,  1.2323232,  1.2564864,  1.2806496,  1.3048128,
        1.328976 ,  1.3531392,  1.3773024,  1.4014656,  1.4256288,
        1.449792 ,  1.4739552,  1.4981184,  1.5222816,  1.5464448,
        1.570608 ,  1.5947712,  1.6189344,  1.6430976,  1.6672608,
        1.691424 ,  1.7155872,  1.7397504,  1.7639136,  1.7880